## Initial setup

Picks GPU, prepares file structure and initializes variables

In [1]:
import tensorflow as tf
import os
import sys

physical_devices = tf.config.list_physical_devices('GPU')
print("Avaliable GPUs:")
for device in physical_devices:
    print(device.name)
print(f"Chosen GPU: {physical_devices[-1].name}")
tf.config.set_visible_devices(physical_devices[-1], 'GPU')

path_common= os.path.dirname(os.path.abspath('')) + "/common/"
sys.path.append(path_common)
print("Current working directory:", os.getcwd())

model, hpo_search_function, hpo_n_epochs_max, hpo_max_trials, hpo_batch_size = [None] * 5

2024-03-14 16:50:49.317174: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-03-14 16:50:49.336322: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-14 16:50:49.336334: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-14 16:50:49.336852: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-03-14 16:50:49.340197: I tensorflow/core/platform/cpu_feature_guar

Avaliable GPUs:
/physical_device:GPU:0
Chosen GPU: /physical_device:GPU:0
Current working directory: /home/slagarp/Master-Thesis/src/main


2024-03-14 16:50:50.081259: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-14 16:50:50.092575: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-14 16:50:50.092606: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.


In [2]:
from ModelTrainer import log_tensorboard_averages, get_log_dir
log_tensorboard_averages(get_log_dir("00"), get_log_dir("00", tag="averages"))


2024-03-14 16:50:51.239988: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-14 16:50:51.240068: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-14 16:50:51.240089: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-14 16:50:51.298854: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-14 16:50:51.298890: I external/local_xla/xla/stream_executor

[('train_loss', 1.8261603116989136, 2), ('val_loss', 1.867999792098999, 2), ('train_accuracy', 0.2908099293708801, 2), ('val_accuracy', 0.28439241647720337, 2), ('runtime', 67.57685852050781, 2), ('train_loss', 1.857558012008667, 1), ('val_loss', 1.8637677431106567, 1), ('train_accuracy', 0.2741672694683075, 1), ('val_accuracy', 0.2835426330566406, 1), ('runtime', 46.743064880371094, 1), ('train_loss', 1.9314236640930176, 0), ('val_loss', 1.882468581199646, 0), ('train_accuracy', 0.25282689929008484, 0), ('val_accuracy', 0.2217920869588852, 0), ('runtime', 25.927974700927734, 0)]
{'train_loss': {2: [1.8261603116989136], 1: [1.857558012008667], 0: [1.9314236640930176]}, 'val_loss': {2: [1.867999792098999], 1: [1.8637677431106567], 0: [1.882468581199646]}, 'train_accuracy': {2: [0.2908099293708801], 1: [0.2741672694683075], 0: [0.25282689929008484]}, 'val_accuracy': {2: [0.28439241647720337], 1: [0.2835426330566406], 0: [0.2217920869588852]}, 'runtime': {2: [67.57685852050781], 1: [46.74

2024-03-14 16:50:51.622401: I external/local_tsl/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory


# Pick your model

Available options:

- FCN:                Fully Convultional Neural Network
- ResNet:             Residual Neural Network
- LSTM_FCN:           Long Short Term Memory with Fully Convulutional Network
- Transformer:        ransformer Based model

In [4]:
from ModelFuncs import FCN, ResNet, LSTM_FCN, Transformer
model = FCN()

## Hyper Parameter Optimization

Pick your search function

Available options:
- random:               Random Search
- bayesian:             Bayesian Optimization

In [5]:
hpo_search_function = "random"

Pick remaining tuning parameters

Available Options:
- hpo_n_epochs_max
- hpo_max_trials
- hpo_batch_size

In [10]:
hpo_n_epochs_max = 3
hpo_max_trials = 10
hpo_batch_size = 128

Run hyper parameter tuning

In [11]:
from HyperParameterTuning import Tuner

#%%capture output
hpo_tuner = Tuner(FCN(), n_epochs_max = hpo_n_epochs_max, max_trials = hpo_max_trials, batch_size = hpo_batch_size, search_function=hpo_search_function)
hpo_tuner.tune()
#output.show()

Trial 1 Complete [00h 06m 51s]
val_accuracy: 0.21852276623249053

Best val_accuracy So Far: 0.21852276623249053
Total elapsed time: 00h 06m 51s

Search: Running Trial #2

Value             |Best Value So Far |Hyperparameter
5                 |4                 |num_layers
256               |32                |filters_block_0
5                 |5                 |kernel_size_block_0
256               |256               |filters_block_1
3                 |3                 |kernel_size_block_1
128               |64                |filters_block_2
7                 |7                 |kernel_size_block_2
64                |64                |filters_block_3
7                 |3                 |kernel_size_block_3
256               |32                |filters_block_4
3                 |7                 |kernel_size_block_4
32                |32                |filters_block_5
7                 |5                 |kernel_size_block_5
0.0068352         |0.003045          |lr

Starting fold

KeyboardInterrupt: 

## Train Model

Get best hyper parameters

In [4]:
from HyperParameterTuning import Tuner

tuner_object = Tuner(FCN(), n_epochs_max = hpo_n_epochs_max, max_trials = hpo_max_trials, batch_size = hpo_batch_size, search_function=hpo_search_function)

tuner = tuner_object.get_tuner()

tuner.reload()

best_hps = tuner.get_best_hyperparameters()[0]

best_model = model.build(best_hps)

best_model.summary()



Reloading Tuner from ../../results/FCN/tuner0.json
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input (InputLayer)          [(None, 700, 1)]          0         
                                                                 
 conv1d (Conv1D)             (None, 700, 128)          1024      
                                                                 
 batch_normalization (Batch  (None, 700, 128)          2800      
 Normalization)                                                  
                                                                 
 activation (Activation)     (None, 700, 128)          0         
                                                                 
 conv1d_1 (Conv1D)           (None, 700, 256)          98560     
                                                                 
 batch_normalization_1 (Bat  (None, 700, 256)          2800      
 chNormali

2024-03-12 16:41:02.112923: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-12 16:41:02.112970: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-12 16:41:02.112988: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-12 16:41:02.192847: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-12 16:41:02.192890: I external/local_xla/xla/stream_executor

Train model

In [6]:
from ModelTrainer import ModelTrainer
mt=ModelTrainer(n_epochs_max=10); 
mt.train_es(best_model)

=== Epoch: 0 ===


2024-03-12 16:42:48.758076: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8904
2024-03-12 16:42:48.852730: I external/local_tsl/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2024-03-12 16:42:49.073412: I external/local_tsl/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2024-03-12 16:42:49.767894: I external/local_xla/xla/service/service.cc:168] XLA service 0x7f69584b2630 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-03-12 16:42:49.767920: I external/local_xla/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce GTX 1080, Compute Capability 6.1
2024-03-12 16:42:49.775610: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1710258169.841403   83614 device_compiler.

 88/760 [==>...........................] - ETA: 43s - loss: 2.0012 - accuracy: 0.2048

KeyboardInterrupt: 